In [2]:
import pandas as pd

In [5]:
DATAPATH = '/content/Quran_English.csv' 

df = pd.read_csv(DATAPATH)
df.rename(columns={'Verse':'sentence'}, inplace=True)
print(len(df))
df = df.iloc[:20000]
df.head()

6236


,Name,Surah,Ayat,sentence
0,The Opening,1,1,"In the name of Allah, the Beneficent, the Merc..."
1,The Opening,1,2,"Praise be to Allah, Lord of the Worlds,"
2,The Opening,1,3,"The Beneficent, the Merciful."
3,The Opening,1,4,"Owner of the Day of Judgment,"
4,The Opening,1,5,Thee (alone) we worship; Thee (alone) we ask f...


In [12]:
from nltk.stem import WordNetLemmatizer
import nltk.corpus
import nltk
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re

STOPWORDS = set(stopwords.words('english'))
MIN_WORDS = 4
MAX_WORDS = 600

PATTERN_S = re.compile("\'s")  # matches `'s` from text  
PATTERN_RN = re.compile("\\r\\n") #matches `\r` and `\n`
PATTERN_PUNC = re.compile(r"[^\w\s]") # matches all non 0-9 A-z whitespace 

def clean_text(text):
    """
    Menghilangkan stopwords dan membuat semua case pada kalimat menjadi lower case
    """
    text = text.lower()  # lowercase text
    text = re.sub(PATTERN_S, ' ', text)
    text = re.sub(PATTERN_RN, ' ', text)
    text = re.sub(PATTERN_PUNC, ' ', text)
    return text

def tokenizer(sentence, min_words=MIN_WORDS, max_words=MAX_WORDS, stopwords=STOPWORDS, lemmatize=True):
    """
    Lemmatize, tokenize, crop and remove stop words.
    """
    if lemmatize:
        stemmer = WordNetLemmatizer()
        tokens = [stemmer.lemmatize(w) for w in word_tokenize(sentence)]
    else:
        tokens = [w for w in word_tokenize(sentence)]
    token = [w for w in tokens if (len(w) > min_words and len(w) < max_words
                                                        and w not in stopwords)]
    return tokens    


def clean_sentences(df):
    """
    Menghapus karakter yang tidak relevan  
    mengtokenisasi kata menjadi list sebuah kata
    """
    print('Cleaning sentences...')
    df['clean_sentence'] = df['sentence'].apply(clean_text)
    df['tok_lem_sentence'] = df['clean_sentence'].apply(
        lambda x: tokenizer(x, min_words=MIN_WORDS, max_words=MAX_WORDS, stopwords=STOPWORDS, lemmatize=True))
    return df
    
df = clean_sentences(df)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Cleaning sentences...


In [14]:
print(len(df))
df[['sentence', 'clean_sentence', 'tok_lem_sentence']]

6236


,sentence,clean_sentence,tok_lem_sentence
0,"In the name of Allah, the Beneficent, the Merc...",in the name of allah the beneficent the merc...,"[in, the, name, of, allah, the, beneficent, th..."
1,"Praise be to Allah, Lord of the Worlds,",praise be to allah lord of the worlds,"[praise, be, to, allah, lord, of, the, world]"
2,"The Beneficent, the Merciful.",the beneficent the merciful,"[the, beneficent, the, merciful]"
3,"Owner of the Day of Judgment,",owner of the day of judgment,"[owner, of, the, day, of, judgment]"
4,Thee (alone) we worship; Thee (alone) we ask f...,thee alone we worship thee alone we ask f...,"[thee, alone, we, worship, thee, alone, we, as..."
...,...,...,...
6231,"The King of mankind,",the king of mankind,"[the, king, of, mankind]"
6232,"The God of mankind,",the god of mankind,"[the, god, of, mankind]"
6233,"From the evil of the sneaking whisperer,",from the evil of the sneaking whisperer,"[from, the, evil, of, the, sneaking, whisperer]"
6234,"Who whispereth in the hearts of mankind,",who whispereth in the hearts of mankind,"[who, whispereth, in, the, heart, of, mankind]"


In [15]:
query_sentence = 'of the jinn and of mankind' 

pd.options.display.max_colwidth = 500

In [17]:
def extract_best_indices(m, topk, mask=None):
    """
    Use sum of the cosine distance over all tokens.
    m (np.array): cos matrix of shape (nb_in_tokens, nb_dict_tokens)
    topk (int): number of indices to return (from high to lowest in order)
    """
    # return the sum on all tokens of cosinus for each sentence
    if len(m.shape) > 1:
        cos_sim = np.mean(m, axis=0) 
    else: 
        cos_sim = m
    index = np.argsort(cos_sim)[::-1] # from highest idx to smallest score 
    if mask is not None:
        assert mask.shape == m.shape
        mask = mask[index]
    else:
        mask = np.ones(len(cos_sim))
    mask = np.logical_or(cos_sim[index] != 0, mask) #eliminate 0 cosine distance
    best_index = index[mask][:topk]  
    return best_index

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Adaptasi stop words
token_stop = tokenizer(' '.join(STOPWORDS), lemmatize=False)

# Fit TFIDF
vectorizer = TfidfVectorizer(stop_words=token_stop, tokenizer=tokenizer) 
tfidf_mat = vectorizer.fit_transform(df['sentence'].values) # -> (num_sentences, num_vocabulary)
tfidf_mat.shape

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['doe', 'ha', 'wa'] not in stop_words.
  % sorted(inconsistent)


(6236, 5543)

In [21]:
query_sentence

'of the jinn and of mankind'

In [23]:
def get_recommendations_tfidf(sentence, tfidf_mat):
    
    """ 
    Mengembalikan kalimat data base dalam urutan cosine similarity tertinggi secara relatif untuk masing-masing
    target kalimat.
    """
    # Embed the query sentence
    tokens = [str(tok) for tok in tokenizer(sentence)]
    vec = vectorizer.transform(tokens)
    # Membuat list dengan similarity antara query dan dataset
    mat = cosine_similarity(vec, tfidf_mat)
    # jarak similarity terbaik untuk setiap token independen
    print(mat.shape)
    best_index = extract_best_indices(mat, topk=3)
    return best_index


best_index = get_recommendations_tfidf(query_sentence, tfidf_mat)

display(df[['Name', 'Surah', 'Ayat','sentence']].iloc[best_index])

(6, 6236)


,Name,Surah,Ayat,sentence
6235,Mankind,114,6,Of the jinn and of mankind.
6232,Mankind,114,3,"The God of mankind,"
3945,Those Who Set The Ranks,37,158,"And they imagine kinship between him and the jinn, whereas the jinn know well that they will be brought before (Him)."


In [28]:
import spacy

#!python -m spacy download en_core_web_lg
!python -m spacy download en
#Load pre-trained model
nlp = spacy.load("en") 
# Apply the model to the sentences
df['spacy_sentence'] = df['sentence'].apply(lambda x: nlp(x)) 
# Retrieve the embedded vectors as a matrix 
embed_mat = df['spacy_sentence'].values
embed_mat.shape

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.0 MB 5.2 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


(6236,)

In [29]:
def predict_spacy(model, query_sentence, embed_mat, topk=5):
    """
    Memprediksi the topk sentences setelah men-applying model spacy.
    """
    query_embed = model(query_sentence)
    mat = np.array([query_embed.similarity(line) for line in embed_mat])
    # Mengkeep vector yang memiliki norm
    mat_mask = np.array(
        [True if line.vector_norm else False for line in embed_mat])
    best_index = extract_best_indices(mat, topk=topk, mask=mat_mask)
    return best_index

# Prediksi
predict_spacy(nlp, query_sentence, embed_mat)
display(df[['Name', 'Surah', 'Ayat', 'sentence']].iloc[best_index])

Streaming output truncated to the last 5000 lines.
/usr/lib/python3.7/runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  "__main__", mod_spec)
/usr/lib/python3.7/runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word 

,Name,Surah,Ayat,sentence
6235,Mankind,114,6,Of the jinn and of mankind.
6232,Mankind,114,3,"The God of mankind,"
3945,Those Who Set The Ranks,37,158,"And they imagine kinship between him and the jinn, whereas the jinn know well that they will be brought before (Him)."


In [32]:
from gensim.models.word2vec import Word2Vec

# Create model
word2vec_model = Word2Vec(min_count=0, workers = 8, size=500) 
# Prepare vocab
word2vec_model.build_vocab(df.tok_lem_sentence.values)
# Train
word2vec_model.train(df.tok_lem_sentence.values, total_examples=word2vec_model.corpus_count, epochs=30)

(3056508, 4676940)

In [35]:
def is_word_in_model(word, model):
    """
    Check on individual words ``word`` that it exists in ``model``.
    """
    assert type(model).__name__ == 'KeyedVectors'
    is_in_vocab = word in model.key_to_index.keys()
    return is_in_vocab

def predict_w2v(query_sentence, dataset, model, topk=3):
    query_sentence = query_sentence.split()
    in_vocab_list, best_index = [], [0]*topk
    for w in query_sentence:
        # remove unseen words from query sentence
        if is_word_in_model(w, model.wv):
            in_vocab_list.append(w)
    # Retrieve the similarity between two words as a distance
    if len(in_vocab_list) > 0:
        sim_mat = np.zeros(len(dataset))  # TO DO
        for i, data_sentence in enumerate(dataset):
            if data_sentence:
                sim_sentence = model.wv.n_similarity(
                        in_vocab_list, data_sentence)
            else:
                sim_sentence = 0
            sim_mat[i] = np.array(sim_sentence)
        # Take the five highest norm
        best_index = np.argsort(sim_mat)[::-1][:topk]
    return best_index

# Predict
best_index = predict_w2v(query_sentence, df['tok_lem_sentence'].values, word2vec_model)    
display(df[['Name', 'Surah', 'Ayat', 'sentence']].iloc[best_index])

AssertionError: ignored

In [37]:
#!pip install sentence_transformers
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
corpus_embeddings = model.encode(df.sentence.values, convert_to_tensor=True)
query_embedding = model.encode(query_sentence, convert_to_tensor=True)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 79 kB 3.6 MB/s 
     |████████████████████████████████| 4.2 MB 11.7 MB/s 
     |████████████████████████████████| 1.2 MB 44.5 MB/s 
     |████████████████████████████████| 86 kB 4.9 MB/s 
     |████████████████████████████████| 596 kB 45.7 MB/s 
     |████████████████████████████████| 6.6 MB 37.1 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=a5ffac05f1d62a2887beed015279c96d43d564bcb8e74ca94cfcd04f28534af3
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.69k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/314 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [38]:
import torch

# We use cosine-similarity and torch.topk to find the highest 3 scores
cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
top_results = torch.topk(cos_scores, k=3)

print("\n\n======================\n\n")
print("Query:", query_sentence)
print("\nTop 5 most similar sentences in corpus:")

for score, idx in zip(top_results[0], top_results[1]):
    score = score.cpu().data.numpy() 
    idx = idx.cpu().data.numpy()
    display(df[['Name', 'Surah', 'Ayat', 'sentence']].iloc[idx])  





Query: of the jinn and of mankind

Top 5 most similar sentences in corpus:


Name                            Mankind
Surah                               114
Ayat                                  6
sentence    Of the jinn and of mankind.
Name: 6235, dtype: object

Name                                                                                                      Those Who Set The Ranks
Surah                                                                                                                          37
Ayat                                                                                                                          158
sentence    And they imagine kinship between him and the jinn, whereas the jinn know well that they will be brought before (Him).
Name: 3945, dtype: object

Name                                                                                                                                  The Wind-Curved Sandhills
Surah                                                                                                                                                        46
Ayat                                                                                                                                                         18
sentence    Such are those on whom the Word concerning nations of the jinn and mankind which have passed away before them hath effect. Lo! they are the losers.
Name: 4527, dtype: object

In [39]:
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoTokenizer, AutoModel, pipeline
import torch
import numpy as np
from tqdm import tqdm


BERT_BATCH_SIZE = 4
MODEL_NAME = 'sentence-transformers/paraphrase-MiniLM-L6-v2'

class BertModel:
    def __init__(self, model_name, device=-1, small_memory=True, batch_size=BERT_BATCH_SIZE):
        self.model_name = model_name
        self._set_device(device)
        self.small_device = 'cpu' if small_memory else self.device
        self.batch_size = batch_size
        self.load_pretrained_model()

    def _set_device(self, device):
        if device == -1 or device == 'cpu':
            self.device = 'cpu'
        elif device == 'cuda' or device == 'gpu':
            self.device = 'cuda'
        elif isinstance(device, int) or isinstance(device, float):
            self.device = 'cuda'
        else:  # default
            self.device = torch.device(
                "cuda" if torch.cuda.is_available() else "cpu")

    def load_pretrained_model(self):
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
        self.model = AutoModel.from_pretrained(self.model_name)
        device = -1 if self.device == 'cpu' else 0
        self.pipeline = pipeline('feature-extraction',
                                 model=self.model, tokenizer=self.tokenizer, device=device)

    def embed(self, data):
        """ Create the embedded matrice from original sentences """
        nb_batchs = 1 if (len(data) < self.batch_size) else len(
            data) // self.batch_size
        batchs = np.array_split(data, nb_batchs)
        mean_pooled = []
        for batch in tqdm(batchs, total=len(batchs), desc='Training...'):
            mean_pooled.append(self.transform(batch))
        mean_pooled_tensor = torch.tensor(
            len(data), dtype=float).to(self.small_device)
        mean_pooled = torch.cat(mean_pooled, out=mean_pooled_tensor)
        self.embed_mat = mean_pooled

    @staticmethod
    def mean_pooling(model_output, attention_mask):
        token_embeddings = model_output[0]
        input_mask_expanded = attention_mask.unsqueeze(
            -1).expand(token_embeddings.size()).float()
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

    def transform(self, data):
        if 'str' in data.__class__.__name__:
            data = [data]
        data = list(data)
        token_dict = self.tokenizer(
            data,
            padding=True,
            truncation=True,
            max_length=512,
            return_tensors="pt")
        token_dict = self.to(token_dict, self.device)
        with torch.no_grad():
            token_embed = self.model(**token_dict)
        # each of the 512 token has a 768 or 384-d vector depends on model)
        attention_mask = token_dict['attention_mask']
        # average pooling of masked embeddings
        mean_pooled = self.mean_pooling(
            token_embed, attention_mask)
        mean_pooled = mean_pooled.to(self.small_device)
        return mean_pooled
    
    def to(self, data: dict, device: str):
        """Send all values to device by calling v.to(device)"""
        data = {k: v.to(device) for k, v in data.items()}
        return data

    def predict(self, in_sentence, topk=3):
        input_vec = self.transform(in_sentence)
        mat = cosine_similarity(input_vec, self.embed_mat)
        # best cos sim for each token independantly
        best_index = extract_best_indices(mat, topk=topk)
        return best_index

In [40]:
# CPU training
bert_model = BertModel(model_name=MODEL_NAME, batch_size=BERT_BATCH_SIZE)
bert_model.embed(df.sentence.values)

Downloading:   0%|          | 0.00/314 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/86.7M [00:00<?, ?B/s]

Training...: 100%|██████████| 1559/1559 [02:50<00:00,  9.12it/s]


In [ ]:
# GPU training
bert_model_gpu = BertModel(model_name=MODEL_NAME, batch_size=BERT_BATCH_SIZE, device='cuda')
bert_model_gpu.transform(df.sentence.values)

In [43]:
query_sentence = 'As for the Disbelievers, Whether thou warn them or thou warn them not it is all one for them'
indices = bert_model_gpu.predict(query_sentence)
display(df[['Name', 'Surah', 'Ayat', 'sentence']].iloc[indices])

NameError: ignored